# Necessary Imports and Settings

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import copy

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


In [33]:
names_1 = ["John", "Harry", "Andrew", "Lisa", "Mary"]
names_2 = ["Henry", "David", "Sophia", "Olivia", "Emma"]
cities_1 = ["London", "Paris", "Oslo", "Istanbul", "Beijing"]
cities_2 = ["Sydney", "Cairo", "Seoul", "Rome", "Prague"]

class DataSample_OneSentence():
    def __init__(self, name_1, city_1):
        self.name_1 = name_1
        self.city_1 = city_1
        self.prev_context = name_1 + " travelled to " + city_1 + "."
        self.current_context = name_1 + " met Lucas. Lucas was 30 years old. Where is " + name_1 + "?"
    def add_prev_encoding(self, encoding):
        self.prev_encoding = encoding
    def add_current_encoding(self, encoding):
        self.curr_encoding = encoding
        
data_points_one_sentence = []
for name_1 in names_1 + names_2:
    for city_1 in cities_1 + cities_2:
        sample = DataSample_OneSentence(name_1, city_1)
        prev_context = sample.prev_context 
        current_context = sample.current_context
        
        prev_tokens = tokenizer.encode(prev_context, return_tensors="pt").to(model.encoder.device)

        with torch.no_grad():
            encoded_prev = model.encoder(prev_tokens)
        
        sample.add_prev_encoding(encoded_prev)
        
        curr_tokens = tokenizer.encode(current_context, return_tensors="pt").to(model.encoder.device)

        with torch.no_grad():
            encoded_curr = model.encoder(curr_tokens)
        
        sample.add_current_encoding(encoded_curr)
        
        data_points_one_sentence.append(sample)


In [6]:
inp_prev = tokenizer.encode("London", return_tensors="pt")
inp_curr = tokenizer.encode("John is visiting", return_tensors="pt")
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt")
entity_embeddings = model.encoder(inp_prev)
curr_embeddings = model.encoder(inp_curr)

curr_embeddings.last_hidden_state = torch.cat((entity_embeddings[0], curr_embeddings[0]), dim=1)
model_out = model(decoder_input_ids=decoder_input_ids, 
                  encoder_outputs=curr_embeddings)

In [13]:
entity_embeddings[0].shape[1]

2

In [4]:
len(data_points_one_sentence)

100

In [10]:
for point in data_points_one_sentence:
    print(point.prev_context)
    print(point.current_context)
    print()

John travelled to London.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Paris.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Oslo.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Istanbul.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Beijing.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Sydney.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Cairo.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Seoul.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Rome.
John met Lucas. Lucas was 30 years old. Where is John?

John travelled to Prague.
John met Lucas. Lucas was 30 years old. Where is John?

Harry travelled to London.
Harry met Lucas. Lucas was 30 years old. Where is Harry?

Harry travelled to Paris.
Harry met Lucas. Lucas was 30 years old. Where is Harry?

Harry travelled

In [7]:
model_out = model.generate(encoder_outputs=curr_embeddings)

/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/utils.py:1320: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
tokenizer.decode(model_out[0])

'<pad> John is visiting</s>'

# No Entities Moved

In [36]:
correctly_guessed_zero = []

for num_data_point, data_point in enumerate(data_points_one_sentence):
    
    current_context = data_point.current_context
    answer = data_point.city_1
    current_context_ids = tokenizer.encode(current_context, return_tensors="pt").cuda()

    out = model.generate(
            input_ids=current_context_ids,
            max_new_tokens=10,
            #return_dict_in_generate=True,
            #output_scores=True
        ) 

    out_generated = tokenizer.decode(out[0], skip_special_tokens=True)

    if answer in out_generated:
        correctly_guessed_zero.append(1)
    else:
        correctly_guessed_zero.append(0)
        

In [37]:
correctly_guessed_zero

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# One Entity to Move

In [34]:
correctly_guessed = defaultdict(list)

for num_data_point, data_point in enumerate(data_points_one_sentence):
    
    print("NOW PROCESSING:", num_data_point)
    start = time.time()
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        curr_embeddings_copy = copy.deepcopy(curr_embeddings)
        
        curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], curr_embeddings[0]), dim=1)
        model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)
        
        out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)
        
        if answer in out_generated:
            correctly_guessed[i].append(1)
        else:
            correctly_guessed[i].append(0)
        
    print("Time taken:", time.time()-start)
        

NOW PROCESSING: 0
Time taken: 3.5975565910339355
NOW PROCESSING: 1
Time taken: 3.3473923206329346
NOW PROCESSING: 2
Time taken: 3.4686155319213867
NOW PROCESSING: 3
Time taken: 3.4683587551116943
NOW PROCESSING: 4
Time taken: 3.3554599285125732
NOW PROCESSING: 5
Time taken: 3.481775999069214
NOW PROCESSING: 6
Time taken: 3.3424971103668213
NOW PROCESSING: 7
Time taken: 3.6465365886688232
NOW PROCESSING: 8
Time taken: 3.5083811283111572
NOW PROCESSING: 9
Time taken: 3.7722439765930176
NOW PROCESSING: 10
Time taken: 3.5165510177612305
NOW PROCESSING: 11
Time taken: 3.4491145610809326
NOW PROCESSING: 12
Time taken: 3.4552712440490723
NOW PROCESSING: 13
Time taken: 4.223069429397583
NOW PROCESSING: 14
Time taken: 3.6872105598449707
NOW PROCESSING: 15
Time taken: 3.636688470840454
NOW PROCESSING: 16
Time taken: 3.6207382678985596
NOW PROCESSING: 17
Time taken: 3.700260877609253
NOW PROCESSING: 18
Time taken: 3.3401904106140137
NOW PROCESSING: 19
Time taken: 3.593874931335449
NOW PROCESSING:

In [32]:
for i in correctly_guessed:
    print(i)
    print(sum(correctly_guessed[i])/len(correctly_guessed[i]))

0
0.61
1
0.62
2
0.66
3
0.63
4
0.7
5
0.59
6
0.6


In [35]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/one_entity_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed, f)

# Two Entities to Move

In [36]:
correctly_guessed_two = {}
my_context = []

for i in range(num_prev_tokens):
    correctly_guessed_two[i] = defaultdict(list)

for data_point in data_points_one_sentence:
    start = time.time()
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        for j in range(i+1, num_prev_tokens):
            
            curr_embeddings_copy = copy.deepcopy(curr_embeddings)
            
            curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                           entity_embeddings[0][:,j:j+1,:], 
                                                           curr_embeddings[0]), dim=1)
            model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

            out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)

            if answer in out_generated:
                correctly_guessed_two[i][j].append(1)
            else:
                correctly_guessed_two[i][j].append(0)

    print("Time taken:", time.time()-start)


Time taken: 9.674528121948242
Time taken: 9.808419942855835
Time taken: 10.724555015563965
Time taken: 10.68189549446106
Time taken: 10.594213008880615
Time taken: 10.570935249328613
Time taken: 10.086668014526367
Time taken: 10.469334125518799
Time taken: 9.964863777160645
Time taken: 11.636019468307495
Time taken: 9.610822916030884
Time taken: 10.077605247497559
Time taken: 10.032696962356567
Time taken: 11.446326494216919
Time taken: 10.948538780212402
Time taken: 10.314511060714722
Time taken: 9.555182695388794
Time taken: 10.713384628295898
Time taken: 9.832491636276245
Time taken: 11.763803005218506
Time taken: 10.679234504699707
Time taken: 10.695507526397705
Time taken: 11.315159559249878
Time taken: 11.317360877990723
Time taken: 10.927242994308472
Time taken: 10.830505609512329
Time taken: 10.439494609832764
Time taken: 11.062764167785645
Time taken: 9.698684215545654
Time taken: 12.19505763053894
Time taken: 8.612093210220337
Time taken: 9.454864263534546
Time taken: 9.57118

In [34]:
for cont in my_context:
    print(tokenizer.decode(cont[0]))

John  John met Lucas. Lucas was 30 years old. Where is John?</s>
Johntravelled John met Lucas. Lucas was 30 years old. Where is John?</s>
John to John met Lucas. Lucas was 30 years old. Where is John?</s>
John London John met Lucas. Lucas was 30 years old. Where is John?</s>
John. John met Lucas. Lucas was 30 years old. Where is John?</s>
John</s> John met Lucas. Lucas was 30 years old. Where is John?</s>
travelled John met Lucas. Lucas was 30 years old. Where is John?</s>
 to John met Lucas. Lucas was 30 years old. Where is John?</s>
 London John met Lucas. Lucas was 30 years old. Where is John?</s>
. John met Lucas. Lucas was 30 years old. Where is John?</s>
</s> John met Lucas. Lucas was 30 years old. Where is John?</s>
travelled to John met Lucas. Lucas was 30 years old. Where is John?</s>
travelled London John met Lucas. Lucas was 30 years old. Where is John?</s>
travelled. John met Lucas. Lucas was 30 years old. Where is John?</s>
travelled</s> John met Lucas. Lucas was 30 years 

In [22]:
for i in range(7):
    for j in range(i+1, 7):
        print(i, j)
        print(sum(correctly_guessed_two[i][j]))
        print(len(correctly_guessed_two[i][j]))
        print(sum(correctly_guessed_two[i][j])/len(correctly_guessed_two[i][j]))
        print()

0 1
58
100
0.58

0 2
60
100
0.6

0 3
59
100
0.59

0 4
71
100
0.71

0 5
68
100
0.68

0 6
64
100
0.64

1 2
69
100
0.69

1 3
71
100
0.71

1 4
75
100
0.75

1 5
72
100
0.72

1 6
74
100
0.74

2 3
80
100
0.8

2 4
81
100
0.81

2 5
82
100
0.82

2 6
81
100
0.81

3 4
84
100
0.84

3 5
86
100
0.86

3 6
87
100
0.87

4 5
89
100
0.89

4 6
90
100
0.9

5 6
88
100
0.88



In [24]:
for i in range(7):
    #for j in range(i+1, 7):
    print(len(correctly_guessed_two[i]))

6
5
4
3
2
1
0


In [37]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/two_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_two, f)

# Three Entities to Move

In [7]:
num_prev_tokens = 7

7

In [8]:
correctly_guessed_three = {}
for i in range(num_prev_tokens):

    correctly_guessed_three[i] = {}

    for j in range(i+1, num_prev_tokens):

        correctly_guessed_three[i][j] = defaultdict(list)

for data_point in data_points_one_sentence:
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        for j in range(i+1, num_prev_tokens):
            
            for k in range(j+1, num_prev_tokens):
                
                curr_embeddings_copy = copy.deepcopy(curr_embeddings)
            
                curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                               entity_embeddings[0][:,j:j+1,:], 
                                                               entity_embeddings[0][:,k:k+1,:], 
                                                               curr_embeddings[0]), dim=1)
                model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

                out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)
                
                if answer in out_generated:
                    correctly_guessed_three[i][j][k].append(1)
                else:
                    correctly_guessed_three[i][j][k].append(0)


In [9]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/three_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_three, f)

# Four Entities to Move

In [10]:
correctly_guessed_four = {}
for i in range(num_prev_tokens):

    correctly_guessed_four[i] = {}

    for j in range(i+1, num_prev_tokens):

        correctly_guessed_four[i][j] = {}

        for k in range(j+1, num_prev_tokens):

            correctly_guessed_four[i][j][k] = defaultdict(list)

for data_point in data_points_one_sentence:
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        for j in range(i+1, num_prev_tokens):
            
            for k in range(j+1, num_prev_tokens):
                
                for l in range(k+1, num_prev_tokens):
                    
                    curr_embeddings_copy = copy.deepcopy(curr_embeddings)
            
                    curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                   entity_embeddings[0][:,j:j+1,:], 
                                                                   entity_embeddings[0][:,k:k+1,:], 
                                                                   entity_embeddings[0][:,l:l+1,:], 
                                                                   curr_embeddings[0]), dim=1)
                
                    model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

                    out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)

                    if answer in out_generated:
                        correctly_guessed_four[i][j][k][l].append(1)
                    else:
                        correctly_guessed_four[i][j][k][l].append(0)


In [11]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/four_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_four, f)

# Five Entities to Move

In [12]:
correctly_guessed_five = {}

for i in range(num_prev_tokens):

    correctly_guessed_five[i] = {}

    for j in range(i+1, num_prev_tokens):

        correctly_guessed_five[i][j] = {}

        for k in range(j+1, num_prev_tokens):

            correctly_guessed_five[i][j][k] = {}

            for l in range(k+1, num_prev_tokens):

                correctly_guessed_five[i][j][k][l] = defaultdict(list)

for data_point in data_points_one_sentence:
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        for j in range(i+1, num_prev_tokens):
            
            for k in range(j+1, num_prev_tokens):
                
                for l in range(k+1, num_prev_tokens):
                    
                    for m in range(l+1, num_prev_tokens):
                        
                        curr_embeddings_copy = copy.deepcopy(curr_embeddings)
                        
                        curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                       entity_embeddings[0][:,j:j+1,:], 
                                                                       entity_embeddings[0][:,k:k+1,:], 
                                                                       entity_embeddings[0][:,l:l+1,:], 
                                                                       entity_embeddings[0][:,m:m+1,:], 
                                                                       curr_embeddings[0]), dim=1)
                
                        model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

                        out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)

                        if answer in out_generated:
                            correctly_guessed_five[i][j][k][l][m].append(1)
                        else:
                            correctly_guessed_five[i][j][k][l][m].append(0)


In [13]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/five_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_five, f)

# Six Entities to Move

In [14]:
correctly_guessed_six = {}
for i in range(num_prev_tokens):
        
    correctly_guessed_six[i] = {}

    for j in range(i+1, num_prev_tokens):

        correctly_guessed_six[i][j] = {}

        for k in range(j+1, num_prev_tokens):

            correctly_guessed_six[i][j][k] = {}

            for l in range(k+1, num_prev_tokens):

                correctly_guessed_six[i][j][k][l] = {}

                for m in range(l+1, num_prev_tokens):

                    correctly_guessed_six[i][j][k][l][m] = defaultdict(list)

for data_point in data_points_one_sentence:
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    for i in range(num_prev_tokens):
        
        for j in range(i+1, num_prev_tokens):
            
            for k in range(j+1, num_prev_tokens):
                
                for l in range(k+1, num_prev_tokens):
                    
                    for m in range(l+1, num_prev_tokens):
                        
                        for n in range(m+1, num_prev_tokens):
                            
                            curr_embeddings_copy = copy.deepcopy(curr_embeddings)

                            curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                           entity_embeddings[0][:,j:j+1,:], 
                                                                           entity_embeddings[0][:,k:k+1,:], 
                                                                           entity_embeddings[0][:,l:l+1,:], 
                                                                           entity_embeddings[0][:,m:m+1,:], 
                                                                           entity_embeddings[0][:,n:n+1,:], 
                                                                           curr_embeddings[0]), dim=1)
                
                            model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

                            out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)

                            if answer in out_generated:
                                correctly_guessed_six[i][j][k][l][m][n].append(1)
                            else:
                                correctly_guessed_six[i][j][k][l][m][n].append(0)

In [15]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/six_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_six, f)

# Seven Entities to Move

In [16]:

correctly_guessed_seven = []

for data_point in data_points_one_sentence:
    prev_context = data_point.prev_context
    current_context = data_point.current_context
    answer = data_point.city_1
    entity_embeddings = data_point.prev_encoding
    curr_embeddings = data_point.curr_encoding
    
    num_prev_tokens = entity_embeddings[0].shape[1]
    
    curr_embeddings_copy = copy.deepcopy(curr_embeddings)
    
    curr_embeddings_copy.last_hidden_state = torch.cat((entity_embeddings[0], 
                                                   curr_embeddings[0]), dim=1)

    model_out = model.generate(encoder_outputs=curr_embeddings_copy, max_new_tokens=10)

    out_generated = tokenizer.decode(model_out[0], skip_special_tokens=True)

    if answer in out_generated:
        correctly_guessed_seven.append(1)
    else:
        correctly_guessed_seven.append(0)

        
        

In [17]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/seven_entities_moved.txt", "wb") as f:
    pickle.dump(correctly_guessed_seven, f)

# All Results

In [18]:
lookup_tokens = {
    0: "John",
    1: "_",
    2: "travelled",
    3: "to",
    4: "London",
    5: ".",
    6: "</s>"
}

In [38]:
all_results = []

for i in range(7):
    my_list = correctly_guessed[i]
    avg_list = sum(my_list)/len(my_list)
    all_results.append(([lookup_tokens[i]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John'], 0.0),
 (['_'], 0.0),
 (['.'], 0.0),
 (['</s>'], 0.0),
 (['travelled'], 0.01),
 (['to'], 0.01),
 (['London'], 0.56)]

In [39]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        my_list = correctly_guessed_two[i][j]
        avg_list = sum(my_list)/len(my_list)
        all_results.append(([lookup_tokens[i], lookup_tokens[j]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_'], 0.0),
 (['John', 'travelled'], 0.0),
 (['John', '.'], 0.0),
 (['John', '</s>'], 0.0),
 (['_', '.'], 0.0),
 (['_', '</s>'], 0.0),
 (['.', '</s>'], 0.0),
 (['John', 'to'], 0.01),
 (['_', 'travelled'], 0.01),
 (['_', 'to'], 0.01),
 (['travelled', 'to'], 0.01),
 (['travelled', '.'], 0.01),
 (['travelled', '</s>'], 0.01),
 (['to', '.'], 0.01),
 (['to', '</s>'], 0.01),
 (['John', 'London'], 0.5),
 (['_', 'London'], 0.54),
 (['London', '</s>'], 0.54),
 (['London', '.'], 0.57),
 (['travelled', 'London'], 0.62),
 (['to', 'London'], 0.62)]

In [20]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            my_list = correctly_guessed_three[i][j][k]
            avg_list = sum(my_list)/len(my_list)
            all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', '.'], 0.0),
 (['John', '_', '</s>'], 0.0),
 (['John', 'travelled', '.'], 0.0),
 (['John', '.', '</s>'], 0.0),
 (['_', '.', '</s>'], 0.0),
 (['John', '_', 'travelled'], 0.01),
 (['John', '_', 'to'], 0.01),
 (['John', 'travelled', '</s>'], 0.01),
 (['John', 'to', '.'], 0.01),
 (['John', 'to', '</s>'], 0.01),
 (['_', 'travelled', '.'], 0.01),
 (['_', 'travelled', '</s>'], 0.01),
 (['_', 'to', '.'], 0.01),
 (['_', 'to', '</s>'], 0.01),
 (['travelled', 'to', '</s>'], 0.01),
 (['travelled', '.', '</s>'], 0.01),
 (['to', '.', '</s>'], 0.01),
 (['_', 'travelled', 'to'], 0.02),
 (['travelled', 'to', '.'], 0.02),
 (['John', 'travelled', 'to'], 0.03),
 (['John', 'London', '.'], 0.48),
 (['John', '_', 'London'], 0.52),
 (['John', 'London', '</s>'], 0.52),
 (['John', 'to', 'London'], 0.56),
 (['_', 'London', '.'], 0.56),
 (['_', 'London', '</s>'], 0.56),
 (['London', '.', '</s>'], 0.56),
 (['to', 'London', '</s>'], 0.59),
 (['John', 'travelled', 'London'], 0.6),
 (['_', 'to', 'Londo

In [21]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                my_list = correctly_guessed_four[i][j][k][l]
                avg_list = sum(my_list)/len(my_list)
                all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', '.', '</s>'], 0.0),
 (['_', 'travelled', '.', '</s>'], 0.0),
 (['_', 'to', '.', '</s>'], 0.0),
 (['John', '_', 'travelled', 'to'], 0.01),
 (['John', '_', 'travelled', '</s>'], 0.01),
 (['John', '_', 'to', '.'], 0.01),
 (['John', '_', 'to', '</s>'], 0.01),
 (['John', 'travelled', 'to', '</s>'], 0.01),
 (['John', 'travelled', '.', '</s>'], 0.01),
 (['John', 'to', '.', '</s>'], 0.01),
 (['_', 'travelled', 'to', '.'], 0.01),
 (['_', 'travelled', 'to', '</s>'], 0.01),
 (['travelled', 'to', '.', '</s>'], 0.01),
 (['John', '_', 'travelled', '.'], 0.02),
 (['John', 'travelled', 'to', '.'], 0.02),
 (['John', 'London', '.', '</s>'], 0.5),
 (['John', '_', 'London', '.'], 0.52),
 (['_', 'London', '.', '</s>'], 0.53),
 (['John', '_', 'London', '</s>'], 0.55),
 (['John', 'to', 'London', '</s>'], 0.56),
 (['John', '_', 'to', 'London'], 0.58),
 (['John', '_', 'travelled', 'London'], 0.59),
 (['John', 'travelled', 'London', '</s>'], 0.59),
 (['John', 'to', 'London', '.'], 0.59),
 (['_',

In [22]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    my_list = correctly_guessed_five[i][j][k][l][m]
                    avg_list = sum(my_list)/len(my_list)
                    all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l], lookup_tokens[m]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'travelled', '.', '</s>'], 0.0),
 (['John', '_', 'travelled', 'to', '</s>'], 0.01),
 (['John', '_', 'to', '.', '</s>'], 0.01),
 (['_', 'travelled', 'to', '.', '</s>'], 0.01),
 (['John', '_', 'travelled', 'to', '.'], 0.02),
 (['John', 'travelled', 'to', '.', '</s>'], 0.02),
 (['John', '_', 'London', '.', '</s>'], 0.52),
 (['John', 'to', 'London', '.', '</s>'], 0.55),
 (['John', '_', 'to', 'London', '.'], 0.57),
 (['John', '_', 'to', 'London', '</s>'], 0.57),
 (['John', '_', 'travelled', 'London', '.'], 0.58),
 (['John', 'travelled', 'London', '.', '</s>'], 0.59),
 (['John', '_', 'travelled', 'London', '</s>'], 0.6),
 (['_', 'to', 'London', '.', '</s>'], 0.6),
 (['_', 'travelled', 'London', '.', '</s>'], 0.62),
 (['John', '_', 'travelled', 'to', 'London'], 0.64),
 (['John', 'travelled', 'to', 'London', '.'], 0.65),
 (['John', 'travelled', 'to', 'London', '</s>'], 0.65),
 (['_', 'travelled', 'to', 'London', '</s>'], 0.67),
 (['travelled', 'to', 'London', '.', '</s>'], 0.68

In [23]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    for n in range(m+1, 7):
                        my_list = correctly_guessed_six[i][j][k][l][m][n]
                        avg_list = sum(my_list)/len(my_list)
                        all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l], lookup_tokens[m], lookup_tokens[n]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'travelled', 'to', '.', '</s>'], 0.01),
 (['John', '_', 'travelled', 'London', '.', '</s>'], 0.56),
 (['John', '_', 'to', 'London', '.', '</s>'], 0.57),
 (['John', '_', 'travelled', 'to', 'London', '.'], 0.63),
 (['John', 'travelled', 'to', 'London', '.', '</s>'], 0.63),
 (['John', '_', 'travelled', 'to', 'London', '</s>'], 0.64),
 (['_', 'travelled', 'to', 'London', '.', '</s>'], 0.65)]

In [25]:
all_results = []
my_list = correctly_guessed_seven
avg_list = sum(my_list)/len(my_list)
all_results.append(([lookup_tokens[0], lookup_tokens[1], lookup_tokens[2], lookup_tokens[3], lookup_tokens[4], lookup_tokens[5], lookup_tokens[6]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'travelled', 'to', 'London', '.', '</s>'], 0.63)]

In [38]:

all_results = []

for i in range(7):
    my_list = correctly_guessed[i]
    avg_list = sum(my_list)/len(my_list)
    all_results.append(([i], avg_list))

for i in range(7):
    for j in range(i+1, 7):
        my_list = correctly_guessed_two[i][j]
        avg_list = sum(my_list)/len(my_list)
        all_results.append(([i, j], avg_list))
        
for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            my_list = correctly_guessed_three[i][j][k]
            avg_list = sum(my_list)/len(my_list)
            all_results.append(([i, j, k], avg_list))

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                my_list = correctly_guessed_four[i][j][k][l]
                avg_list = sum(my_list)/len(my_list)
                all_results.append(([i, j, k, l], avg_list))
                
for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    my_list = correctly_guessed_five[i][j][k][l][m]
                    avg_list = sum(my_list)/len(my_list)
                    all_results.append(([i, j, k, l, m], avg_list))

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    for n in range(m+1, 7):
                        my_list = correctly_guessed_six[i][j][k][l][m][n]
                        avg_list = sum(my_list)/len(my_list)
                        all_results.append(([i, j, k, l, m, n], avg_list))
                        
avg_seven = sum(correctly_guessed_seven)/len(correctly_guessed_seven)
all_results.append(([0, 1, 2, 3, 4, 5, 6], avg_seven))

avg_zero = sum(correctly_guessed_zero)/len(correctly_guessed_zero)
all_results.append(([], avg_zero))

In [39]:
len(all_results)

128

In [40]:
all_results

[([0], 0.0),
 ([1], 0.0),
 ([2], 0.01),
 ([3], 0.01),
 ([4], 0.64),
 ([5], 0.63),
 ([6], 0.63),
 ([0, 1], 0.58),
 ([0, 2], 0.6),
 ([0, 3], 0.59),
 ([0, 4], 0.71),
 ([0, 5], 0.68),
 ([0, 6], 0.64),
 ([1, 2], 0.69),
 ([1, 3], 0.71),
 ([1, 4], 0.75),
 ([1, 5], 0.72),
 ([1, 6], 0.74),
 ([2, 3], 0.8),
 ([2, 4], 0.81),
 ([2, 5], 0.82),
 ([2, 6], 0.81),
 ([3, 4], 0.84),
 ([3, 5], 0.86),
 ([3, 6], 0.87),
 ([4, 5], 0.89),
 ([4, 6], 0.9),
 ([5, 6], 0.88),
 ([0, 1, 2], 0.86),
 ([0, 1, 3], 0.89),
 ([0, 1, 4], 0.87),
 ([0, 1, 5], 0.87),
 ([0, 1, 6], 0.85),
 ([0, 2, 3], 0.86),
 ([0, 2, 4], 0.89),
 ([0, 2, 5], 0.89),
 ([0, 2, 6], 0.92),
 ([0, 3, 4], 0.92),
 ([0, 3, 5], 0.92),
 ([0, 3, 6], 0.94),
 ([0, 4, 5], 0.94),
 ([0, 4, 6], 0.94),
 ([0, 5, 6], 0.94),
 ([1, 2, 3], 0.94),
 ([1, 2, 4], 0.94),
 ([1, 2, 5], 0.93),
 ([1, 2, 6], 0.93),
 ([1, 3, 4], 0.94),
 ([1, 3, 5], 0.95),
 ([1, 3, 6], 0.94),
 ([1, 4, 5], 0.93),
 ([1, 4, 6], 0.93),
 ([1, 5, 6], 0.94),
 ([2, 3, 4], 0.93),
 ([2, 3, 5], 0.94),
 ([2, 3, 6

In [41]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/Case 2/all_results.txt", "wb") as f:
    pickle.dump(all_results, f)

In [42]:
all_results.sort(key=lambda x: x[1])

In [43]:
all_results

[([0], 0.0),
 ([1], 0.0),
 ([], 0.0),
 ([2], 0.01),
 ([3], 0.01),
 ([0, 1], 0.58),
 ([0, 3], 0.59),
 ([0, 2], 0.6),
 ([5], 0.63),
 ([6], 0.63),
 ([4], 0.64),
 ([0, 6], 0.64),
 ([0, 5], 0.68),
 ([1, 2], 0.69),
 ([0, 4], 0.71),
 ([1, 3], 0.71),
 ([1, 5], 0.72),
 ([1, 6], 0.74),
 ([1, 4], 0.75),
 ([2, 3], 0.8),
 ([2, 4], 0.81),
 ([2, 6], 0.81),
 ([2, 5], 0.82),
 ([3, 4], 0.84),
 ([0, 1, 6], 0.85),
 ([3, 5], 0.86),
 ([0, 1, 2], 0.86),
 ([0, 2, 3], 0.86),
 ([3, 6], 0.87),
 ([0, 1, 4], 0.87),
 ([0, 1, 5], 0.87),
 ([5, 6], 0.88),
 ([4, 5], 0.89),
 ([0, 1, 3], 0.89),
 ([0, 2, 4], 0.89),
 ([0, 2, 5], 0.89),
 ([4, 6], 0.9),
 ([0, 2, 6], 0.92),
 ([0, 3, 4], 0.92),
 ([0, 3, 5], 0.92),
 ([2, 4, 5], 0.92),
 ([1, 2, 5], 0.93),
 ([1, 2, 6], 0.93),
 ([1, 4, 5], 0.93),
 ([1, 4, 6], 0.93),
 ([2, 3, 4], 0.93),
 ([2, 3, 6], 0.93),
 ([2, 5, 6], 0.93),
 ([3, 5, 6], 0.93),
 ([4, 5, 6], 0.93),
 ([0, 1, 3, 5], 0.93),
 ([0, 3, 6], 0.94),
 ([0, 4, 5], 0.94),
 ([0, 4, 6], 0.94),
 ([0, 5, 6], 0.94),
 ([1, 2, 3], 0.

In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)